<a href="https://www.bigdatauniversity.com"><img src="https://ibm.box.com/shared/static/cw2c7r3o20w9zn8gkecaeyjhgw3xdgbj.png" width="400" align="center"></a>

<h1 align="center"><font size="5">Classification with Python</font></h1>

**Lets first load required libraries:**

In [1]:
#Basic Libraries
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing
%matplotlib inline

#Libraries need to model build-up
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

#Libraries for scoring
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss

**About dataset**

This dataset is about past loans. The __Loan_train.csv__ data set includes details of 346 customers whose loan are already paid off or defaulted. It includes following fields:

| Field          | Description                                                                           |
|----------------|---------------------------------------------------------------------------------------|
| Loan_status    | Whether a loan is paid off on in collection                                           |
| Principal      | Basic principal loan amount at the                                                    |
| Terms          | Origination terms which can be weekly (7 days), biweekly, and monthly payoff schedule |
| Effective_date | When the loan got originated and took effects                                         |
| Due_date       | Since it’s one-time payoff schedule, each loan has one single due date                |
| Age            | Age of applicant                                                                      |
| Education      | Education of applicant                                                                |
| Gender         | The gender of applicant                                                               |

**Lets download the dataset**

In [2]:
!wget -O loan_train.csv https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_train.csv

--2020-02-27 09:09:12--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_train.csv
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23101 (23K) [text/csv]
Saving to: ‘loan_train.csv’

100%[======================================>] 23,101      --.-K/s   in 0.002s  

2020-02-27 09:09:12 (12.6 MB/s) - ‘loan_train.csv’ saved [23101/23101]



**Load Data From CSV File**

In [3]:
df = pd.read_csv('loan_train.csv')
df.head()

,Unnamed: 0,Unnamed: 0.1,loan_status,Principal,terms,effective_date,due_date,age,education,Gender
0,0,0,PAIDOFF,1000,30,9/8/2016,10/7/2016,45,High School or Below,male
1,2,2,PAIDOFF,1000,30,9/8/2016,10/7/2016,33,Bechalor,female
2,3,3,PAIDOFF,1000,15,9/8/2016,9/22/2016,27,college,male
3,4,4,PAIDOFF,1000,30,9/9/2016,10/8/2016,28,college,female
4,6,6,PAIDOFF,1000,30,9/9/2016,10/8/2016,29,college,male


**Feature Engineering and Train/Test Split**

In [4]:
X=df.drop(columns='loan_status')
y=df.loan_status
num_feat=X.select_dtypes(include='number').columns.to_list()
cat_feat=X.select_dtypes(include='object').columns.to_list()
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.2, random_state=42)

**Pipeline Buliding**

In [5]:
num_pipe=Pipeline([
    ('scaler', StandardScaler())
])

cat_pipe=Pipeline([
    ('encoder',OneHotEncoder(handle_unknown='ignore'))
])

ct=ColumnTransformer(remainder='drop',
                    transformers=[
                        ('numerical', num_pipe, num_feat),
                        ('categorical',cat_pipe, cat_feat)
                    ])

# Decision Tree Classifier

In [6]:
model_dt=Pipeline([
    ('transformer',ct),
    ('predictor',DecisionTreeClassifier())
])
model_dt.fit(X_train, y_train);

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/base.py:467: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)


In [7]:
y_pred_test=model_dt.predict(X_test)
y_pred_train=model_dt.predict(X_train)
print('Accuracy score on train set: ', metrics.accuracy_score(y_train, y_pred_train))
print('Accuracy score on test set: ', metrics.accuracy_score(y_test, y_pred_test))

Accuracy score on train set:  1.0
Accuracy score on test set:  1.0


/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


**This seems over ambitious!**

# K-Nearest Neighbour Classifier

In [8]:
model_kn=Pipeline([
    ('transformer',ct),
    ('predictor',KNeighborsClassifier(n_neighbors=4))
])
model_kn.fit(X_train, y_train);

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/base.py:467: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)


In [9]:
y_pred_test=model_kn.predict(X_test)
y_pred_train=model_kn.predict(X_train)
print('Accuracy score on train set: ', metrics.accuracy_score(y_train, y_pred_train))
print('Accuracy score on test set: ', metrics.accuracy_score(y_test, y_pred_test))

Accuracy score on train set:  0.9963768115942029
Accuracy score on test set:  1.0


/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


**Not bad at all!**

# Support Vector Machine Classifier

In [10]:
model_sv=Pipeline([
    ('transformer', ct),
    ('predictor',SVC())
])
model_sv.fit(X_train, y_train);

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/base.py:467: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [11]:
y_pred_test=model_sv.predict(X_test)
y_pred_train=model_sv.predict(X_train)
print('Accuracy score on train set: ', metrics.accuracy_score(y_train, y_pred_train))
print('Accuracy score on test set: ', metrics.accuracy_score(y_test, y_pred_test))

Accuracy score on train set:  1.0
Accuracy score on test set:  1.0


/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


**Great result indeed! But the question is: is this overfitting?, In my thinking, NO**

# Logistic Regression Classifier

In [12]:
model_lr=Pipeline([
    ('transformer',ct),
    ('predictor',LogisticRegression())
])
model_lr.fit(X_train, y_train);

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/base.py:467: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [13]:
y_pred_test=model_lr.predict(X_test)
y_pred_train=model_lr.predict(X_train)
print('Accuracy score on train set: ', metrics.accuracy_score(y_train, y_pred_train))
print('Accuracy score on test set: ', metrics.accuracy_score(y_test, y_pred_test))

Accuracy score on train set:  1.0
Accuracy score on test set:  1.0


/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


**Ah! are you kidding me? really, I am surprised with accuracy score!**

# Model Evaluation using Test set

**First, download and load the test set:**

In [14]:
!wget -O loan_test.csv https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_test.csv

--2020-02-27 09:09:14--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_test.csv
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3642 (3.6K) [text/csv]
Saving to: ‘loan_test.csv’

100%[======================================>] 3,642       --.-K/s   in 0s      

2020-02-27 09:09:14 (422 MB/s) - ‘loan_test.csv’ saved [3642/3642]



**Load Test set for evaluation**

In [15]:
test_df = pd.read_csv('loan_test.csv')
X_test=test_df.drop(columns='loan_status')
y_test=test_df.loan_status

In [16]:
y_kn=model_kn.predict(X_test);
y_dt=model_dt.predict(X_test);
y_sv=model_sv.predict(X_test);
y_lr=model_lr.predict(X_test);

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


In [17]:
jaccard_score=[
    jaccard_similarity_score(y_kn, y_test),
    jaccard_similarity_score(y_dt, y_test),
    jaccard_similarity_score(y_sv, y_test),
    jaccard_similarity_score(y_lr, y_test)
]

F1_score=[
    f1_score(y_kn, y_test, average='weighted'),
    f1_score(y_dt, y_test,average='weighted'),
    f1_score(y_sv, y_test, average='weighted'),
    f1_score(y_lr, y_test, average='weighted')
]

y_lr_1=(y_lr=='PAIDOFF')
y_test_1=(y_test=='PAIDOFF')
logloss_score=['NA','NA','NA',log_loss(y_lr_1, y_test_1)
    ]


In [18]:
data={'Algorithm':['KNN','Decision Tree','SVM','Logistic Regression'],\
      'Jaccard':jaccard_score, 'F1-score':F1_score, 'Log Loss':logloss_score}
result=pd.DataFrame(data=data, columns=['Algorithm','Jaccard','F1-score','Log Loss'], index=None)
result

,Algorithm,Jaccard,F1-score,Log Loss
0,KNN,0.962963,0.962963,NA
1,Decision Tree,1.000000,1.000000,NA
2,SVM,1.000000,1.000000,NA
3,Logistic Regression,1.000000,1.000000,9.99201e-16


**Good job done, Congratulations!**